<h1><center>ЛОГИРОВАНИЕ ФАЙЛА В УДАЛЁННОМ ХРАНИЛИЩЕ</h1>

Вам нужно поднять такой сервис, чтобы оптимизировать работу вашей команды. Обратите внимание, что для работы с базой данных Postgresql вам понадобится библиотека psycopg. Установить ее можно с помощью такой команды: pip install psycopg && pip install 'psycopg[binary]'. Мы подробнее расскажем об этой библиотеке в следующем уроке.
<br><br>Напишите скрипт запуска:
- Создайте новый файл **run_mlflow_server.sh** для локального запуска Tracking Server с удалённой базой данных PostgreSQL и удалённым объектным хранилищем Object Storage в Yandex Cloud.

In [1]:
# export MLFLOW_S3_ENDPOINT_URL=https://storage.yandexcloud.net
# export AWS_ACCESS_KEY_ID=$AWS_ACCESS_KEY_ID
# export AWS_SECRET_ACCESS_KEY=$AWS_SECRET_ACCESS_KEY
# export AWS_BUCKET_NAME=$S3_BUCKET_NAME

# mlflow server \
#   --backend-store-uri postgresql://$DB_DESTINATION_USER:$DB_DESTINATION_PASSWORD@$DB_DESTINATION_HOST:$DB_DESTINATION_PORT/$DB_DESTINATION_NAME\
#     --default-artifact-root s3://$AWS_BUCKET_NAME \
#     --no-serve-artifacts 

- Дайте старт Tracking Server, запустив ваш скрипт через консоль с помощью такой команды: sh run_mlflow_server.sh.

In [2]:
#  sh run_mlflow_server.sh

- Создайте новую тетрадку в Jupyter Notebook, в которой вы будете логировать тестовые артефакты для проверки работоспособности вашего сервиса:

In [4]:
import os
import mlflow

# определяем основные credentials, которые нужны для подключения к MLflow
# важно, что credentials мы передаём для себя как пользователей Tracking Service
# у вас должен быть доступ к бакету, в который вы будете складывать артефакты
os.environ["MLFLOW_S3_ENDPOINT_URL"] = "https://storage.yandexcloud.net" #endpoint бакета от YandexCloud
os.environ["AWS_ACCESS_KEY_ID"] = 'YCAJE3Nlz8iDILW5VTYM1ihQB' # os.getenv("AWS_ACCESS_KEY_ID") # получаем id ключа бакета, к которому подключён MLFlow, из .env
os.environ["AWS_SECRET_ACCESS_KEY"] = 'YCPjvS7uwhvJpUj3bKm8X-IX4QAwBIVsvX61IL44' # os.getenv("AWS_SECRET_ACCESS_KEY") #  получаем ключ бакета, к которому подключён MLFlow, из .env

# определяем глобальные переменные
# поднимаем MLflow локально
TRACKING_SERVER_HOST = "127.0.0.1"
TRACKING_SERVER_PORT = 5000

YOUR_NAME = "unique_experiment-2" # введите своё имя для создания уникального эксперимента
assert YOUR_NAME, "alexnikzotov"

# название тестового эксперимента и запуска (run) внутри него
EXPERIMENT_NAME = f"test_connection_experiment_{YOUR_NAME}"
RUN_NAME = "test_connection_run"

# тестовые данные
METRIC_NAME = "test_metric"
METRIC_VALUE = 0

# устанавливаем host, который будет отслеживать наши эксперименты
mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")

# создаём тестовый эксперимент и записываем в него тестовую информацию
experiment_id = mlflow.create_experiment(EXPERIMENT_NAME)
with mlflow.start_run(run_name=RUN_NAME, experiment_id=experiment_id) as run:
    run_id = run.info.run_id
    
    mlflow.log_metric(METRIC_NAME, METRIC_VALUE) 

Далее запустите следующий код, который проверит, что всё сделано правильно:

In [5]:
##### 4. Проверяем себя, что в MLflow:
# - создался `experiment` с нашим именем
# - внутри эксперимента появился запуск `run`
# - внутри `run` записалась наша тестовая `metric`
experiment = mlflow.get_experiment_by_name(EXPERIMENT_NAME)
run = mlflow.get_run(run_id)

assert "active" == experiment.lifecycle_stage
assert mlflow.get_run(run_id)
assert METRIC_VALUE == run.data.metrics[METRIC_NAME] 

In [ ]:
# import sqlalchemy
# from sqlalchemy import create_engine
# import pandas as pd

# connection = {"sslmode": "require", "target_session_attrs": "read-write"}
# postgres_credentials = { "host": "rc1b-uh7kdmcx67eomesf.mdb.yandexcloud.net", 
#                          "port": "6432",
#                          "dbname": "playground_mle_20250529_05fed48463",
#                          "user": "mle_20250529_05fed48463",
#                          "password": "0c567edd8ad8472e87d5c85cc4d664e4" }
# assert all([var_value != "" for var_value in list(postgres_credentials.values())])

# # Создаем строку подключения
# connection_string = f"postgresql://{postgres_credentials['user']}:{postgres_credentials['password']}@{postgres_credentials['host']}:{postgres_credentials['port']}/{postgres_credentials['dbname']}"

# # Определяем название таблицы
# TABLE_NAME = "users_churn"

# # Устанавливаем соединение с базой данных
# engine = create_engine(connection_string)

# # Выполняем SQL-запрос и загружаем данные в DataFrame
# df = pd.read_sql_query(f"SELECT * FROM {TABLE_NAME}", engine)

ModuleNotFoundError: No module named 'psycopg2'

In [ ]:
# import psycopg
# connection = {"sslmode": "require", "target_session_attrs": "read-write"}

# postgres_credentials = { "host": "rc1b-uh7kdmcx67eomesf.mdb.yandexcloud.net", 
#                          "port": "6432",
#                          "dbname": "playground_mle_20250529_05fed48463",
#                          "user": "mle_20250529_05fed48463",
#                          "password": "0c567edd8ad8472e87d5c85cc4d664e4" }
# assert all([var_value != "" for var_value in list(postgres_credentials.values())])

# connection.update(postgres_credentials)

# # определим название таблицы, в которой хранятся наши данные.
# TABLE_NAME = "users_churn"

# # эта конструкция создаёт контекстное управление для соединения с базой данных 
# # оператор with гарантирует, что соединение будет корректно закрыто после выполнения всех операций 
# # закрыто оно будет даже в случае ошибки, чтобы не допустить "утечку памяти"
# with psycopg.connect(**connection) as conn:

# # создаёт объект курсора для выполнения запросов к базе данных
# # с помощью метода execute() выполняется SQL-запрос для выборки данных из таблицы TABLE_NAME
#     with conn.cursor() as cur:
#         cur.execute(f"SELECT * FROM {TABLE_NAME}")
                
#                 # извлекаем все строки, полученные в результате выполнения запроса
#         data = cur.fetchall()

#                 # получает список имён столбцов из объекта курсора
#         columns = [col[0] for col in cur.description]

# # создаёт объект DataFrame из полученных данных и имён столбцов. 
# # это позволяет удобно работать с данными в Python, используя библиотеку Pandas.
# df = pd.DataFrame(data, columns=columns) 

ModuleNotFoundError: No module named 'psycopg'